OpenAI API(챗 완성) 사용해보기

In [ ]:
!pip install openai==0.28 # openai 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 699.8 kB/s eta 0:00:00


In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import json

# 개발자가 연동을 하고 싶은 함수를 정의한다.
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit = "celsius"):

    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }

    return json.dumps(weather_info)

In [ ]:
ret = get_current_weather('서울')

In [ ]:
print(ret)

{"location": "\uc11c\uc6b8", "temperature": "24", "unit": "celsius", "forecast": ["sunny", "windy"]}


In [ ]:
def get_gps():
    return '서울'

In [ ]:
# 함수 호출을 위한 dict

available_functions = {
    "get_current_weather": get_current_weather,
    "get_location_from_gps" : get_gps, # 일부러 이름을 다르게 함 (이유: 이해를 돕기 위해)
}

# dict (사전)
# 사과(키) : 빨간색의 ... (밸류)
# 바나나(키) : 노란색의 ...(밸류)

# 사용설명서 : LLM에게 넘길 함수명 기재
# dict key : LLM이 특정했을 때, 받을 함수명
# 함수명을 받았으면, 그 이름에 해당하는 함수를 호출해야기 때문에
# 이 매칭을 위한 dict가 필요하다.

In [ ]:
ret = available_functions["get_current_weather"]('서울')

# ret = get_current_weather('서울')

print(ret)

{"location": "\uc11c\uc6b8", "temperature": "24", "unit": "celsius", "forecast": ["sunny", "windy"]}


In [ ]:
ret = available_functions["get_location_from_gps"]()

# ret = get_gps()

print(ret)

서울


In [ ]:
def run_conversation(input_text):
    # Step 1: send the conversation and available functions to GPT
    messages = [
        {"role": "user", "content": "input_text"}
    ]

    # 함수 설명서를 작성 for LLM
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location"],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-0613",
        messages = messages,
        functions = functions,
        function_call = "auto",  # auto is default, but we'll be explicit
    )

    print(response)

    # 함수 선정을 했는 지 안했는 지는 어떻게 알지?

    response_message = response["choices"][0]["message"]

    # 함수 특정에 성공하면 "function_call" 키가 존재합니다.
    print(response_message)

    # function_call키가 있으면 contents = null
    # contents = xxx function_call키가 없음
    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"): # 함수 특정에 성공했다면,
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        # 함수 이름과 인자를 얻어온다.
        function_name = response_message["function_call"]["name"]
        function_args = json.loads(response_message["function_call"]["arguments"])

        # response_message : function_call, name, arguments
        # 함수 이름과 인자 정보를 담은 메시지를 추가한다.
        messages.append(response_message)  # extend conversation with assistant's reply

        # 함수를 호출해서 그 결과를 얻는다.
        function_response = available_functions[function_name](
            location = function_args.get("location"),
            unit = function_args.get("unit"),
        )

        # 함수 호출 결과를 메시지에 추가한다.
        messages.append(
            {
                "role": "function", # 역할 : 함수 호출 결과는 역할을 function으로 설정한다.
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("## messages ##")
        print(messages)
        print("## messages ##")

        second_response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo-0613",
            messages = messages, # 사용자 요청사항, 함수 이름 및 인자, 함수 호출 결과
        )  # get a new response from GPT where it can see the function response

        second_response = second_response["choices"][0]["message"]

        return second_response
    else:
        return response

In [ ]:
response = run_conversation("hello")

print(response)

{
  "id": "chatcmpl-9c73XuCq8MNCmQOjdsnI61m3LizBs",
  "object": "chat.completion",
  "created": 1718870615,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hi there! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 11,
    "total_tokens": 86
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": "Hi there! How can I assist you today?"
}
{
  "id": "chatcmpl-9c73XuCq8MNCmQOjdsnI61m3LizBs",
  "object": "chat.completion",
  "created": 1718870615,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hi there! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 11,
    "

In [ ]:
response = run_conversation("What's the weather in Seoul?")

print(response)

{
  "id": "chatcmpl-9c73Y9PqSGPnxGqFMZn2bm4gVbLnV",
  "object": "chat.completion",
  "created": 1718870616,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Seoul\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 81,
    "completion_tokens": 17,
    "total_tokens": 98
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\n}"
  }
}
## messages ##
[{'role': 'user', 'content': "What's the weather in Seoul?"}, <OpenAIObject at 0x7f4ceff70680> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"locat

In [ ]:
response = run_conversation("What's the weather?")
print(response)

{
  "id": "chatcmpl-9c73adF4ZlezwKpLl2eQYRjxRz7AF",
  "object": "chat.completion",
  "created": 1718870618,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "In order to provide you with the current weather, could you please let me know the location you are interested in?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 24,
    "total_tokens": 103
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": "In order to provide you with the current weather, could you please let me know the location you are interested in?"
}
{
  "id": "chatcmpl-9c73adF4ZlezwKpLl2eQYRjxRz7AF",
  "object": "chat.completion",
  "created": 1718870618,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "In order to provide you with t

In [ ]:
def run_conversation(user_input_text):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": user_input_text}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
        {
            "name": "get_location_from_gps",
            "description": "Get the current location",
            "parameters": {
                "type": "object",
                "properties": {
                    "unit": {"type": "string"},
                },
                "required": [],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-0613",
        messages = messages,
        functions = functions,
        function_call = "auto",  # auto is default, but we'll be explicit
    )

    print(response)

    response_message = response["choices"][0]["message"]

    print(response_message)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        # 함수 이름과 함수 인자를 얻는다.
        function_name = response_message["function_call"]["name"]
        function_args = json.loads(response_message["function_call"]["arguments"])

        # 함수 호출
        # 어떤 함수가 호출되는 지 모르기 때문에, 즉 인자를 특정할 수 없다.
        function_response = available_functions[function_name](**function_args)

        # [중요!!] 함수 이름과 인자 정보 추가
        messages.append(response_message)  # extend conversation with assistant's reply

        # 함수 호출 결과 추가 (role : function)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("## messages ##")
        print(messages)
        print("## messages ##")

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response

        second_response = second_response["choices"][0]["message"]

        return second_response
    else:
        return response_message

In [ ]:
message = run_conversation("현재 위치는 어디인가요?")

print(message.content)

{
  "id": "chatcmpl-9c73bzrIS2HLf8Ac98dhuaAnD3m37",
  "object": "chat.completion",
  "created": 1718870619,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_location_from_gps",
          "arguments": "{}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 108,
    "completion_tokens": 9,
    "total_tokens": 117
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}
## messages ##
[{'role': 'user', 'content': '현재 위치는 어디인가요?'}, <OpenAIObject at 0x7f4ceff71da0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}, {'role': 'function', 'name': 'get_location_from_gps', 'content': '서

In [ ]:
message = run_conversation("서울의 날씨는요?")

{
  "id": "chatcmpl-9c73dcBwqVIYViMFgxmCfVlQ7qgyz",
  "object": "chat.completion",
  "created": 1718870621,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Seoul\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 109,
    "completion_tokens": 17,
    "total_tokens": 126
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\n}"
  }
}
## messages ##
[{'role': 'user', 'content': '서울의 날씨는요?'}, <OpenAIObject at 0x7f4ceff73c90> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\

In [ ]:
message.content

'서울의 현재 날씨는 맑고 바람이 강하며, 온도는 24도입니다.'

In [ ]:
response = run_conversation("안녕하세요.")

{
  "id": "chatcmpl-9c73egwMDRqgnkM1oWMZDDIuwPzrT",
  "object": "chat.completion",
  "created": 1718870622,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\uc548\ub155\ud558\uc138\uc694! \ub3c4\uc6c0\uc774 \ud544\uc694\ud558\uc2e0\uac00\uc694?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 103,
    "completion_tokens": 19,
    "total_tokens": 122
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": "\uc548\ub155\ud558\uc138\uc694! \ub3c4\uc6c0\uc774 \ud544\uc694\ud558\uc2e0\uac00\uc694?"
}


In [ ]:
response.content

'안녕하세요! 도움이 필요하신가요?'

In [ ]:
response = run_conversation("현재 날씨는?")

{
  "id": "chatcmpl-9c73eNbFcFY3IhOSUdMPKHrownSGa",
  "object": "chat.completion",
  "created": 1718870622,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_location_from_gps",
          "arguments": "{}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 106,
    "completion_tokens": 9,
    "total_tokens": 115
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}
## messages ##
[{'role': 'user', 'content': '현재 날씨는?'}, <OpenAIObject at 0x7f4ceff2cf90> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}, {'role': 'function', 'name': 'get_location_from_gps', 'content': '서울'}]
#

In [ ]:
response.content

'현재 서울의 날씨는 어떤가요?'

In [ ]:
run_conversation("What's the weather in Seoul?")

{
  "id": "chatcmpl-9c7V8n53BJO9BoI3MEpb40l957o10",
  "object": "chat.completion",
  "created": 1718872326,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Seoul\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 104,
    "completion_tokens": 17,
    "total_tokens": 121
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\n}"
  }
}
## messages ##
[{'role': 'user', 'content': "What's the weather in Seoul?"}, <OpenAIObject at 0x7f4ceff90770> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"loc

<OpenAIObject at 0x7f4ceff901d0> JSON: {
  "role": "assistant",
  "content": "The current weather in Seoul is sunny and windy with a temperature of 24 degrees Celsius."
}